<a href="https://colab.research.google.com/github/SHINGSkywallker/test/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup

import google.generativeai as genai

# Google Auth & Sheets 相關套件
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

# Google 認證與 Gspread 設定
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

# 初始化 gspread 客戶端
gc = gspread.authorize(creds)

# 從 Colab Secrets 獲取 GOOGLE_API_KEY 並配置 genai
from google.colab import userdata
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)
model = genai.GenerativeModel('gemini-2.5-flash')

# Google Sheet 相關設定
SHEET_URL = "https://docs.google.com/spreadsheets/d/1seJsvAgNfAOQhfAzd5_8Xo28owKUnlTPAbDM5yDQLgU/edit?gid=0#gid=0"
WORKSHEET_NAME = "工作表1" # 主要工作表名稱
TIMEZONE = "Asia/Taipei" # 時區設定

# 確保 Google Sheet 存在，若不存在則建立
def ensure_spreadsheet(name):
    """
    檢查並確保指定名稱的 Google Sheet 存在。
    Args:
        name (str): Google Sheet 的名稱。
    Returns:
        gspread.models.Spreadsheet: 對應的 Spreadsheet 物件。
    """
    try:
        sh = gc.open(name)  # returns gspread.models.Spreadsheet
    except gspread.SpreadsheetNotFound:
        sh = gc.create(name)
    return sh

# 確保工作表 (worksheet) 存在並有正確的表頭
def ensure_worksheet(sh, title, header):
    """
    檢查並確保指定標題的工作表存在於給定的 Spreadsheet 中，並有正確的表頭。
    若不存在則建立，若表頭不符則清除並寫入新的表頭。
    Args:
        sh (gspread.models.Spreadsheet): 目標 Spreadsheet 物件。
        title (str): 目標工作表的標題。
        header (list): 工作表應有的表頭列表。
    Returns:
        gspread.models.Worksheet: 對應的工作表物件。
    """
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(header)+5))
        ws.update([header])
    # 若沒有表頭就補上
    data = ws.get_all_values()
    if not data or (data and data[0] != header):
        ws.clear()
        ws.update([header])
    return ws

# 定義各工作表的表頭
TASKS_HEADER = [
    "id","task","status","priority","est_min","start_time","end_time",
    "actual_min","pomodoros","due_date","labels","notes",
    "created_at","updated_at","completed_at","planned_for"
]
LOGS_HEADER = [
    "log_id","task_id","phase","start_ts","end_ts","minutes","cycles","note"
]
CLIPS_HEADER = ["clip_id","url","selector","text","href","created_at","added_to_task"]

# 獲取當前時區時間
def tznow():
    """
    獲取當前設定時區的時間。
    Returns:
        datetime.datetime: 帶有時區資訊的當前時間。
    """
    return dt.now(gettz(TIMEZONE))

# 從工作表讀取資料並轉換為 DataFrame
def read_df(ws, header):
    """
    從指定工作表讀取資料，轉換為 DataFrame，處理缺失值、確保欄位齊全並微調型別。
    Args:
        ws (gspread.models.Worksheet): 要讀取的工作表物件。
        header (list): 預期的 DataFrame 欄位表頭。
    Returns:
        pd.DataFrame: 從工作表讀取的 DataFrame.
    """
    df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    if df is None or df.empty:
        return pd.DataFrame(columns=header)
    df = df.fillna("")
    # 保證欄位齊全
    for c in header:
        if c not in df.columns:
            df[c] = ""
    # 型別微調，將數字欄位轉換為整數，錯誤值轉為0
    if "est_min" in df.columns:
        df["est_min"] = pd.to_numeric(df["est_min"], errors="coerce").fillna(0).astype(int)
    if "actual_min" in df.columns:
        df["actual_min"] = pd.to_numeric(df["actual_min"], errors="coerce").fillna(0).astype(int)
    if "pomodoros" in df.columns:
        df["pomodoros"] = pd.to_numeric(df["pomodoros"], errors="coerce").fillna(0).astype(int)
    return df[header] # 只保留預期的欄位

# 將 DataFrame 寫入工作表
def write_df(ws, df, header):
    """
    將 DataFrame 的資料寫入指定工作表，會保留現有表頭並追加資料。
    Args:
        ws (gspread.models.Worksheet): 要寫入的工作表物件。
        df (pd.DataFrame): 要寫入的 DataFrame。
        header (list): 工作表的表頭。
    """
    if df.empty:
        ws.clear()
        ws.update([header])
        return
    # 轉字串避免 gspread 型別問題
    df_out = df.copy()
    for c in df_out.columns:
        df_out[c] = df_out[c].astype(str)

    # Get existing data, preserving header
    existing_data = ws.get_all_values()
    if existing_data and existing_data[0] == header:
        existing_header = existing_data[0]
        existing_rows = existing_data[1:]
    else:
        # If no header or incorrect header, clear and write new header
        ws.clear()
        ws.update([header])
        existing_header = header
        existing_rows = []

    # Prepare new rows to append
    new_rows = df_out.values.tolist()

    # Combine existing rows and new rows
    all_rows = existing_rows + new_rows

    # Write combined data back to the sheet
    # This approach appends new data while keeping existing data
    ws.update([existing_header] + all_rows)

# 產生任務選擇列表，用於 Gradio 下拉選單
def list_task_choices():
    """
    根據 tasks_df 產生用於 Gradio Dropdown 的任務選項列表。
    Returns:
        list: 格式為 [(顯示文字, 任務ID), ...] 的列表。
    """
    global tasks_df
    if tasks_df.empty:
        return []
    # 顯示格式： [狀態] (P:優先級) 任務名稱 — 任務ID
    def row_label(r):
        return f"[{r['status']}] (P:{r['priority']}) {r['task']} — {r['id']}"
    return [(row_label(r), r["id"]) for _, r in tasks_df.iterrows()]

# 根據番茄鐘紀錄重新計算任務的實際花費時間和番茄數
def recalc_task_actuals(task_id):
    """
    根據 logs_df 中屬於該任務的工作階段紀錄，重新計算任務的 actual_min 與 pomodoros，並更新 tasks_df。
    Args:
        task_id (str): 要重新計算的任務 ID。
    """
    global tasks_df, logs_df
    # 過濾出該任務的工作階段紀錄
    work_logs = logs_df[(logs_df["task_id"]==task_id) & (logs_df["phase"]=="work")]
    # 計算總分鐘數
    total_min = work_logs["minutes"].astype(float).sum() if not work_logs.empty else 0
    # 計算番茄數 (每 25 分鐘算一個番茄)
    pomos = int(round(total_min / 25.0))
    # 找到任務在 tasks_df 中的索引並更新欄位
    idx = tasks_df.index[tasks_df["id"]==task_id]
    if len(idx)==0: return
    i = idx[0]
    tasks_df.loc[i,"actual_min"] = int(total_min)
    tasks_df.loc[i,"pomodoros"] = pomos
    tasks_df.loc[i,"updated_at"] = tznow().isoformat() # 更新修改時間

# 計算今日任務完成率
def today_summary():
    """
    計算並回報今日計畫任務的總數、完成數以及完成率。
    Returns:
        str: 包含今日任務摘要的文字。
    """
    global tasks_df
    today = tznow().date().isoformat()
    # 過濾出今日計畫的任務
    planned = tasks_df[
        ((tasks_df["due_date"]==today) | (tasks_df["planned_for"].str.lower()=="today"))
    ]
    # 計算已完成的任務
    done = planned[planned["status"]=="done"]
    total = len(planned)
    done_n = len(done)
    # 計算完成率
    rate = (done_n/total*100) if total>0 else 0
    return f"📅 今日計畫任務：{total}；✅ 完成：{done_n}；📈 完成率：{rate:.1f}%"

# 重新整理所有 DataFrame 資料
def refresh_all():
    """
    從 Google Sheet 重新讀取所有工作表資料，更新全局的 DataFrame。
    Returns:
        tuple: 包含 tasks_df, logs_df, clips_df 以及用於 Gradio 更新的列表。
    """
    global tasks_df, logs_df, clips_df
    tasks_df = read_df(ws_tasks, TASKS_HEADER).copy()
    logs_df = read_df(ws_logs, LOGS_HEADER).copy()
    clips_df = read_df(ws_clips, CLIPS_HEADER).copy()
    # 回傳更新後的 DataFrame 以及 Gradio 需要的選擇列表和摘要
    return tasks_df, logs_df, clips_df, list_task_choices(), today_summary(), list_task_choices(), list_task_choices()

# 初始化時從 Sheet 載入資料到 DataFrame
sh = ensure_spreadsheet(WORKSHEET_NAME)
ws_tasks = ensure_worksheet(sh, "tasks", TASKS_HEADER) # 任務工作表
ws_logs  = ensure_worksheet(sh, "pomodoro_logs", LOGS_HEADER) # 番茄鐘紀錄工作表
ws_clips = ensure_worksheet(sh, "web_clips", CLIPS_HEADER) # 爬蟲擷取工作表
tasks_df, logs_df, clips_df = refresh_all()[0:3]


# 新增任務到 tasks_df 和 Google Sheet
def add_task(task, priority, est_min, due_date, labels, notes, planned_for):
    """
    新增一個任務到 tasks_df 並寫回 Google Sheet。
    Args:
        task (str): 任務名稱。
        priority (str): 優先級 (H/M/L)。
        est_min (int): 預估時間（分鐘）。
        due_date (str): 到期日 (YYYY-MM-DD)。
        labels (str): 標籤（逗號分隔）。
        notes (str): 備註。
        planned_for (str): 規劃歸屬 (today/tomorrow/空)。
    Returns:
        tuple: 包含訊息文字和更新後的 tasks_df。
    """
    global tasks_df
    _now = tznow().isoformat()
    new = pd.DataFrame([{
        "id": str(uuid.uuid4())[:8], # 生成短 UUID 作為 ID
        "task": task.strip(),
        "status": "todo", # 預設狀態為 todo
        "priority": priority or "M",
        "est_min": int(est_min) if est_min else 25,
        "start_time": "",
        "end_time": "",
        "actual_min": 0,
        "pomodoros": 0,
        "due_date": due_date or "",
        "labels": labels or "",
        "notes": notes or "",
        "created_at": _now,
        "updated_at": _now,
        "completed_at": "",
        "planned_for": planned_for or ""
    }])
    tasks_df = pd.concat([tasks_df, new], ignore_index=True) # 合併新的任務
    write_df(ws_tasks, tasks_df, TASKS_HEADER) # 寫回 Google Sheet
    return "✅ 已新增任務", tasks_df

# 更新任務狀態
def update_task_status(task_id, new_status):
    """
    更新指定任務 ID 的狀態並寫回 Google Sheet。
    Args:
        task_id (str): 要更新的任務 ID。
        new_status (str): 新的狀態 (todo/in-progress/done).
    Returns:
        tuple: 包含訊息文字和更新後的 tasks_df.
    """
    global tasks_df
    idx = tasks_df.index[tasks_df["id"] == task_id]
    if len(idx)==0:
        return "⚠️ 找不到任務", tasks_df
    i = idx[0]
    tasks_df.loc[i, "status"] = new_status
    tasks_df.loc[i, "updated_at"] = tznow().isoformat()
    if new_status == "done" and not tasks_df.loc[i, "completed_at"]:
        tasks_df.loc[i, "completed_at"] = tznow().isoformat() # 標記完成時間
    write_df(ws_tasks, tasks_df, TASKS_HEADER)
    return "✅ 狀態已更新", tasks_df

# 標記任務為完成
def mark_done(task_id):
    """
    將指定任務標記為完成狀態。
    Args:
        task_id (str): 要標記完成的任務 ID。
    Returns:
        tuple: 包含訊息文字和更新後的 tasks_df。
    """
    return update_task_status(task_id, "done")

# 番茄鐘計時的核心邏輯：記錄開始時間
# 使用全局變數 _active_sessions 儲存進行中的番茄鐘階段資訊
_active_sessions = {}  # { task_id: {"phase": "work"/"break", "start_ts": iso, "cycles": int} }

def start_phase(task_id, phase, cycles):
    """
    開始一個番茄鐘階段 (工作或休息)，記錄開始時間。
    Args:
        task_id (str): 相關的任務 ID。
        phase (str): 階段類型 ('work' 或 'break')。
        cycles (int): 番茄鐘循環次數。
    Returns:
        str: 開始階段的訊息。
    """
    if not task_id: return "⚠️ 請先選擇任務"
    _active_sessions[task_id] = {
        "phase": phase,
        "start_ts": tznow().isoformat(),
        "cycles": int(cycles) if cycles else 1
    }
    return f"▶️ 已開始：{phase}（task: {task_id}）"

# 番茄鐘計時的核心邏輯：計算時間並記錄到 logs_df
def end_phase(task_id, note):
    """
    結束一個番茄鐘階段，計算持續時間，記錄到 logs_df 並寫回 Google Sheet。
    如果是工作階段，會同時更新 tasks_df 的 actual_min 和 pomodoros。
    Args:
        task_id (str): 相關的任務 ID。
        note (str): 階段備註。
    Returns:
        str: 結束階段的訊息。
    """
    global logs_df, tasks_df
    if task_id not in _active_sessions:
        return "⚠️ 尚未開始任何階段"
    # 移除進行中的階段紀錄
    sess = _active_sessions.pop(task_id)
    # 計算持續時間
    start = pd.to_datetime(sess["start_ts"])
    end = tznow()
    minutes = round((end - start).total_seconds() / 60.0, 2)
    # 建立新的番茄鐘紀錄
    log = pd.DataFrame([{
        "log_id": str(uuid.uuid4())[:8], # 生成短 UUID 作為 Log ID
        "task_id": task_id,
        "phase": sess["phase"],
        "start_ts": start.isoformat(),
        "end_ts": end.isoformat(),
        "minutes": minutes,
        "cycles": int(sess["cycles"]),
        "note": note or ""
    }])
    logs_df = pd.concat([logs_df, log], ignore_index=True) # 合併新的紀錄
    write_df(ws_logs, logs_df, LOGS_HEADER) # 寫回 Google Sheet

    # 如果結束的是工作階段，更新對應任務的實際時間和番茄數
    if sess["phase"] == "work":
        recalc_task_actuals(task_id)
        write_df(ws_tasks, tasks_df, TASKS_HEADER) # 寫回 tasks_df

    return f"⏹️ 已結束：{sess['phase']}，紀錄 {minutes} 分鐘"

# AI 計畫功能：根據任務產生今日行動計畫（使用 Gemini 或規則式）
def generate_today_plan():
    """
    根據 tasks_df 中 planned_for 為 'today' 或 due_date 為今天的未完成任務，
    使用 Gemini (若有 API 金鑰) 或規則式方法產生今日行動計畫。
    Returns:
        str: 格式化的今日行動計畫文字（Markdown）。
    """
    global tasks_df
    # 以「due_date 是今天」或「planned_for = today」且不是 done 的任務為計畫清單
    today = tznow().date().isoformat()
    cand = tasks_df[
        ((tasks_df["due_date"]==today) | (tasks_df["planned_for"].str.lower()=="today")) &
        (tasks_df["status"]!="done")
    ].copy()
    if cand.empty:
        return "📭 今天沒有標記的任務。請在 Tasks 分頁把任務的 due_date 設為今天或 planned_for 設為 today。"

    # 先依 priority（H>M>L）+ est_min 排序
    pr_order = {"H":0, "M":1, "L":2}
    cand["p_ord"] = cand["priority"].map(pr_order).fillna(3)
    cand = cand.sort_values(["p_ord","est_min"], ascending=[True, True])

    # 嘗試使用 Gemini 產生計畫
    api_key = os.environ.get("GOOGLE_API_KEY","").strip()
    if api_key:
        genai.configure(api_key=api_key)
        # Gemini 系統提示詞
        sys_prompt = (
            "你是一位任務規劃助理。請把輸入的任務（含估時與優先級）排成三段：morning、afternoon、evening，"
            "並給出每段的重點、順序、每項的時間預估與備註。總時數請大致符合任務估時總和。"
            "回傳以 Markdown 條列，格式：\n"
            "### Morning\n- [任務ID] 任務名稱（預估 xx 分）— 備註\n..."
            "### Afternoon\n...\n### Evening\n...\n"
        )
        # 準備給 Gemini 的任務列表
        items = []
        for _, r in cand.iterrows():
            items.append({
                "id": r["id"], "task": r["task"], "est_min": int(r["est_min"]),
                "priority": r["priority"]
            })
        user_content = json.dumps({"today": today, "tasks": items}, ensure_ascii=False)
        try:
            model = genai.GenerativeModel("gemini-2.5-flash")
            resp = model.generate_content(sys_prompt + "\n\n" + user_content)
            plan_md = resp.text
        except Exception as e:
            # Gemini 失敗時回報錯誤並改用規則式
            plan_md = f"⚠️ Gemini 失敗：{e}\n\n改用規則式規劃。"
    else:
        # 未設定 API 金鑰時使用規則式
        plan_md = "🔧 未設定 GOOGLE_API_KEY，使用規則式規劃。\n\n"

    # 規則式規劃：將任務平均分配到上午/下午/晚上
    buckets = {"morning": [], "afternoon": [], "evening": []}
    total = len(cand)
    for i, (_, r) in enumerate(cand.iterrows()):
        if i % 3 == 0:
            buckets["morning"].append(r)
        elif i % 3 == 1:
            buckets["afternoon"].append(r)
        else:
            buckets["evening"].append(r)

    # 格式化規則式規劃的輸出
    def sec_md(name, rows):
        if not rows: return f"### {name.title()}\n（無）\n"
        lines = [f"### {name.title()}"]
        for r in rows:
            lines.append(f"- [{r['id']}] {r['task']}（預估 {int(r['est_min'])} 分，P:{r['priority']}）")
        return "\n".join(lines) + "\n"

    rule_md = sec_md("morning", buckets["morning"]) + "\n" + \
              sec_md("afternoon", buckets["afternoon"]) + "\n" + \
              sec_md("evening", buckets["evening"])

    # 合併 Gemini (如果成功) 和規則式規劃的結果
    return (plan_md + "\n---\n" + rule_md).strip()


# =========================
# 爬蟲功能：擷取網頁內容
# =========================
def crawl(url, selector, mode, limit):
    """
    從指定 URL 使用 CSS Selector 擷取網頁內容。
    Args:
        url (str): 目標網址。
        selector (str): CSS Selector 字串。
        mode (str): 擷取內容模式 ('text', 'href', 'both')。
        limit (int): 最多擷取筆數。
    Returns:
        tuple: 包含擷取結果的 DataFrame (clips_df 格式) 和訊息文字。
    """
    try:
        # 發送 HTTP 請求獲取網頁內容
        resp = requests.get(url, timeout=15, headers={"User-Agent":"Mozilla/5.0"})
        resp.raise_for_status() # 檢查請求是否成功
    except Exception as e:
        return pd.DataFrame(columns=CLIPS_HEADER), f"⚠️ 請求失敗：{e}"

    # 使用 BeautifulSoup 解析網頁內容
    soup = BeautifulSoup(resp.text, "html.parser")
    # 根據 selector 找到所有匹配的元素
    nodes = soup.select(selector)
    rows = []
    # 遍歷找到的元素，根據 mode 擷取文字和/或連結
    for i, n in enumerate(nodes[:int(limit) if limit else 20]):
        text = n.get_text(strip=True) if mode in ("text","both") else ""
        href = n.get("href") if mode in ("href","both") else ""
        # 處理相對連結，轉換為絕對連結
        if href and href.startswith("/"):
            from urllib.parse import urljoin
            href = urljoin(url, href)
        # 將擷取結果存為字典
        rows.append({
            "clip_id": str(uuid.uuid4())[:8], # 生成短 UUID 作為 Clip ID
            "url": url,
            "selector": selector,
            "text": text,
            "href": href,
            "created_at": tznow().isoformat(),
            "added_to_task": "" # 標記是否已加入任務
        })
    # 將結果轉換為 DataFrame
    df = pd.DataFrame(rows, columns=CLIPS_HEADER)
    return df, f"✅ 擷取 {len(df)} 筆"

# 將爬蟲擷取的項目加入為任務
def add_clips_as_tasks(clip_ids, default_priority, est_min):
    """
    根據提供的 clip_id 列表，將 web_clips 中的項目轉換為新的任務，並寫回 tasks_df 和 Google Sheet。
    Args:
        clip_ids (list): 要轉換為任務的 clip_id 列表。
        default_priority (str): 新增任務的預設優先級。
        est_min (int): 新增任務的預估時間（分鐘）。
    Returns:
        tuple: 包含訊息文字、更新後的 clips_df 和更新後的 tasks_df。
    """
    global clips_df, tasks_df
    if not clip_ids:
        return "⚠️ 請先勾選要加入的爬蟲項目", clips_df, tasks_df
    # 過濾出要加入任務的 clip 項目
    sel = clips_df[clips_df["clip_id"].isin(clip_ids)]
    _now = tznow().isoformat()
    new_tasks = []
    # 遍歷選取的 clip 項目，建立新的任務字典
    for _, r in sel.iterrows():
        title = r["text"] or r["href"] or "（未命名）" # 任務標題取文字或連結
        note = f"來源：{r['url']}\n選擇器：{r['selector']}\n連結：{r['href']}" # 任務備註包含來源資訊
        new_tasks.append({
            "id": str(uuid.uuid4())[:8], # 生成短 UUID 作為任務 ID
            "task": title[:120], # 任務標題截斷至 120 字
            "status": "todo",
            "priority": default_priority or "M",
            "est_min": int(est_min) if est_min else 25,
            "start_time": "",
            "end_time": "",
            "actual_min": 0,
            "pomodoros": 0,
            "due_date": "",
            "labels": "from:crawler", # 自動加入 crawler 標籤
            "notes": note,
            "created_at": _now,
            "updated_at": _now,
            "completed_at": "",
            "planned_for": ""
        })
    if new_tasks:
        tasks_df = pd.concat([tasks_df, pd.DataFrame(new_tasks)], ignore_index=True) # 合併新的任務
        # 標記 clips_df 中已加入任務的項目
        clips_df.loc[clips_df["clip_id"].isin(clip_ids), "added_to_task"] = "yes"
        write_df(ws_tasks, tasks_df, TASKS_HEADER) # 寫回 tasks_df
        write_df(ws_clips, clips_df, CLIPS_HEADER) # 寫回 clips_df
        return f"✅ 已加入 {len(new_tasks)} 項為任務", clips_df, tasks_df
    return "⚠️ 無可加入項目", clips_df, tasks_df

# 刪除任務函數
def delete_task(task_id):
    """
    根據任務 ID 刪除 tasks_df 中的任務並寫回 Google Sheet。
    Args:
        task_id (str): 要刪除的任務 ID。
    Returns:
        tuple: 包含訊息文字和更新後的 tasks_df。
    """
    global tasks_df
    if not task_id:
        return "⚠️ 請先選擇要刪除的任務", tasks_df

    # 找到要刪除任務的索引
    idx = tasks_df.index[tasks_df["id"] == task_id]
    if len(idx) == 0:
        return "⚠️ 找不到任務", tasks_df

    # 刪除任務並重置索引
    tasks_df = tasks_df.drop(idx).reset_index(drop=True)
    write_df(ws_tasks, tasks_df, TASKS_HEADER) # 寫回 Google Sheet
    return "✅ 任務已刪除", tasks_df

# =========================
# Gradio 介面定義
# =========================
# 內部使用的重新整理函數，用於 Gradio 按鈕綁定
def _refresh():
    """
    內部使用的重新整理函數，調用 refresh_all() 並回傳 Gradio 元件所需的輸出。
    """
    # 調用 refresh_all 獲取更新後的 DataFrame 和列表
    tasks_df, logs_df, clips_df, task_choices_list1, summary_text, task_choices_list2, task_choices_list3 = refresh_all()
    # 回傳 Gradio 元件對應的輸出
    return tasks_df, logs_df, clips_df, task_choices_list1, summary_text, task_choices_list2, task_choices_list3


# 定義 Gradio Blocks 介面
with gr.Blocks(title="待辦清單＋番茄鐘＋AI 計畫（Sheet/Gradio/爬蟲）") as demo:
    gr.Markdown("# ✅ 待辦清單與番茄鐘（Google Sheet＋Gradio＋Crawler＋AI 計畫）")
    with gr.Row():
        btn_refresh = gr.Button("🔄 重新整理（Sheet → App）") # 重新整理按鈕
        out_summary = gr.Markdown(today_summary()) # 顯示今日任務摘要

    # 任務管理分頁
    with gr.Tab("Tasks"):
        with gr.Row():
            # 新增任務區塊
            with gr.Column(scale=2):
                task = gr.Textbox(label="任務名稱", placeholder="寫 HW3 報告 / 修正 SQL / …")
                priority = gr.Dropdown(["H","M","L"], value="M", label="優先級")
                est_min = gr.Number(value=25, label="預估時間（分鐘）", precision=0)
                due_date = gr.Textbox(label="到期日（YYYY-MM-DD，可空白）")
                labels = gr.Textbox(label="標籤（逗號分隔，可空白）")
                notes = gr.Textbox(label="備註（可空白）")
                planned_for = gr.Dropdown(["","today","tomorrow"], value="", label="規劃歸屬")
                btn_add = gr.Button("➕ 新增任務")
                msg_add = gr.Markdown() # 新增任務訊息顯示

            # 任務清單顯示區塊
            with gr.Column(scale=3):
                grid_tasks = gr.Dataframe(value=tasks_df, label="任務清單（直接從 Sheet 來）", interactive=False)

        # 更新任務狀態區塊
        with gr.Row():
            task_choice = gr.Dropdown(choices=list_task_choices(), label="選取任務（用於更新）") # 任務選擇下拉選單
            new_status = gr.Dropdown(["todo","in-progress","done"], value="in-progress", label="更新狀態") # 狀態選擇下拉選單
            btn_update = gr.Button("✏️ 更新狀態") # 更新狀態按鈕
            btn_done = gr.Button("✅ 直接標記完成") # 標記完成按鈕
            msg_update = gr.Markdown() # 更新狀態訊息顯示

        # 刪除任務區塊
        with gr.Row():
            delete_task_choice = gr.Dropdown(choices=list_task_choices(), label="選取要刪除的任務") # 任務選擇下拉選單
            btn_delete = gr.Button("🗑️ 刪除任務") # 刪除任務按鈕
            msg_delete = gr.Markdown() # 刪除任務訊息顯示


    # 番茄鐘分頁
    with gr.Tab("Pomodoro"):
        with gr.Row():
            sel_task = gr.Dropdown(choices=list_task_choices(), label="選擇任務") # 任務選擇下拉選單
            cycles = gr.Number(value=1, precision=0, label="番茄數（僅作紀錄）") # 番茄數輸入
        with gr.Row():
            btn_start_work = gr.Button("▶️ 開始工作") # 開始工作按鈕
            note_work = gr.Textbox(label="工作備註（可空白）") # 工作備註輸入
            btn_end_work = gr.Button("⏹️ 結束工作並記錄") # 結束工作按鈕
        with gr.Row():
            btn_start_break = gr.Button("🍵 開始休息") # 開始休息按鈕
            note_break = gr.Textbox(label="休息備註（可空白）") # 休息備註輸入
            btn_end_break = gr.Button("⏹️ 結束休息並記錄") # 結束休息按鈕
        msg_pomo = gr.Markdown() # 番茄鐘訊息顯示
        grid_logs = gr.Dataframe(value=logs_df, label="番茄鐘紀錄", interactive=False) # 番茄鐘紀錄顯示

    # AI 計畫分頁
    with gr.Tab("AI Plan"):
        gr.Markdown("把**今天的任務**排成 **morning / afternoon / evening** 三段行動計畫。若未設 GOOGLE_API_KEY，會用規則式。") # 功能說明
        btn_plan = gr.Button("🧠 產生今日計畫") # 產生計畫按鈕
        out_plan = gr.Markdown() # 計畫輸出顯示

    # 爬蟲分頁
    with gr.Tab("Crawler"):
        # 爬蟲設定區塊
        url = gr.Textbox(label="目標 URL", placeholder="https://example.com") # URL 輸入
        selector = gr.Textbox(label="CSS Selector", placeholder="a.news-item / h2.title / div.card a") # Selector 輸入
        mode = gr.Radio(["text","href","both"], value="text", label="擷取內容") # 擷取模式選擇
        limit = gr.Number(value=20, precision=0, label="最多擷取幾筆") # 擷取數量限制
        btn_crawl = gr.Button("🕷️ 開始擷取") # 開始擷取按鈕
        msg_crawl = gr.Markdown() # 爬蟲訊息顯示
        grid_clips = gr.Dataframe(value=clips_df, label="擷取結果（會同步寫入 Sheet）", interactive=True) # 擷取結果顯示

        # 將擷取項目加入任務區塊
        clip_ids = gr.Textbox(label="要加入任務的 clip_id（多個以逗號分隔）") # 選擇要加入任務的 clip_id
        default_priority = gr.Dropdown(["H","M","L"], value="L", label="新增任務優先級") # 新增任務預設優先級
        clip_est = gr.Number(value=25, precision=0, label="新增任務預估分鐘") # 新增任務預估時間
        btn_add_clips = gr.Button("➕ 將勾選的擷取項目加入為任務") # 加入任務按鈕
        msg_add_clips = gr.Markdown() # 加入任務訊息顯示

    # 摘要分頁
    with gr.Tab("Summary"):
        btn_summary = gr.Button("📊 重新計算今日完成率") # 重新計算摘要按鈕
        out_summary2 = gr.Markdown() # 今日完成率顯示

    # === 綁定 Gradio 元件的事件與函數 ===
    # 重新整理按鈕點擊事件
    btn_refresh.click(_refresh, outputs=[grid_tasks, grid_logs, grid_clips, task_choice, out_summary, delete_task_choice, sel_task])

    # 新增任務按鈕點擊事件
    btn_add.click(
        add_task,
        inputs=[task, priority, est_min, due_date, labels, notes, planned_for],
        outputs=[msg_add, grid_tasks]
    )

    # 更新任務狀態按鈕點擊事件
    btn_update.click(
        update_task_status,
        inputs=[task_choice, new_status],
        outputs=[msg_update, grid_tasks]
    )

    # 直接標記完成按鈕點擊事件
    btn_done.click(
        mark_done,
        inputs=[task_choice],
        outputs=[msg_update, grid_tasks]
    )

    # 刪除任務按鈕點擊事件
    btn_delete.click(
        delete_task,
        inputs=[delete_task_choice],
        outputs=[msg_delete, grid_tasks]
    )

    # 番茄鐘：開始工作按鈕點擊事件
    btn_start_work.click(
        start_phase, inputs=[sel_task, gr.State("work"), cycles], outputs=[msg_pomo]
    )
    # 番茄鐘：結束工作按鈕點擊事件
    btn_end_work.click(
        end_phase, inputs=[sel_task, note_work], outputs=[msg_pomo]
    )
    # 番茄鐘：開始休息按鈕點擊事件
    btn_start_break.click(
        start_phase, inputs=[sel_task, gr.State("break"), cycles], outputs=[msg_pomo]
    )
    # 番茄鐘：結束休息按鈕點擊事件
    btn_end_break.click(
        end_phase, inputs=[sel_task, note_break], outputs=[msg_pomo]
    )

    # AI 計畫：產生計畫按鈕點擊事件
    btn_plan.click(generate_today_plan, outputs=[out_plan])

    # 爬蟲：執行爬蟲並儲存結果到 clips_df 和 Sheet
    def _crawl_and_save(u, s, m, l):
        df, msg = crawl(u, s, m, l)
        # 寫入 web_clips（覆蓋式追加：合併舊資料）
        global clips_df
        if not df.empty:
            clips_df = pd.concat([clips_df, df], ignore_index=True)
            write_df(ws_clips, clips_df, CLIPS_HEADER)
        return msg, clips_df

    # 爬蟲：開始擷取按鈕點擊事件
    btn_crawl.click(_crawl_and_save, inputs=[url, selector, mode, limit], outputs=[msg_crawl, grid_clips])

    # 爬蟲：將選取的 clip 項目加入為任務
    def _add_clips(clip_ids_str, pr, est):
        ids = [c.strip() for c in (clip_ids_str or "").split(",") if c.strip()]
        msg, new_clips, new_tasks = add_clips_as_tasks(ids, pr, est)
        return msg, new_clips, new_tasks

    # 爬蟲：將勾選的擷取項目加入為任務按鈕點擊事件
    btn_add_clips.click(
        _add_clips,
        inputs=[clip_ids, default_priority, clip_est],
        outputs=[msg_add_clips, grid_clips, grid_tasks]
    )

    # 摘要：重新計算今日完成率按鈕點擊事件
    btn_summary.click(today_summary, outputs=[out_summary2])

# 啟動 Gradio 介面
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://dc6a63e7aec0fa8667.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
